# Logitic regression using a MLP

In [1]:
import pickle
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as skms
import sklearn.linear_model as skl
import sklearn.metrics as skm
import tqdm
import copy
import time
from IPython.display import display

%matplotlib inline

from annCopy import NeuralNetMLP

In [2]:
cwd = os.getcwd()
filenames = glob.glob(os.path.join(cwd, 'files*'))

In [3]:
label_filename = "/home/hanna/project2/files/Ising2DFM_reSample_L40_T=All_labels.pkl"

# Read in the labels
with open(label_filename, "rb") as f:
    labels = pickle.load(f)

dat_filename = "/home/hanna/project2/files/Ising2DFM_reSample_L40_T=All.pkl"
    
# Read in the corresponding configurations
with open(dat_filename, "rb") as f:
    data = np.unpackbits(pickle.load(f)).reshape(-1, 1600).astype("int")

# Set spin-down to -1
data[data == 0] = -1

In [4]:
# Set up slices of the dataset
ordered = slice(0, 70000)
critical = slice(70000, 100000)
disordered = slice(100000, 160000)

X = np.concatenate((data[ordered], data[disordered]))
Y = np.concatenate((labels[ordered], labels[disordered]))

In [5]:
#X_train, X_test, y_train, y_test = skms.train_test_split(X,Y, test_size = 0.333)

In [6]:
from utils import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, split_size=0.5, random_state=0)

# Neural net MLP

In [8]:
# Experimental setup
metric = []

eta = [0.0001, 0.001, 0.01, 0.1]
#lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]
epochs = [1, 10, 50, 100]
batchsize = [1, 10, 50]


for epoch in epochs:
    for e in eta:
        for batch in batchsize:
            ann = NeuralNetMLP(n_hidden=30, 
                               epochs=epoch, 
                               eta=e, 
                               shuffle=True,
                               batch_size=batch, 
                               activation='sigmoid', 
                               tpe = "logistic")

            ann.fit(X_train, y_train, X_test, y_test)
            ann.predict(X_test)
            # returns a list of the mean mse score for different epochs or batches
            metric.append(ann.eval_["valid_preform"])
            print("Sigmoid for nr of epochs "+str(epoch) + " and eta: " + str(e) + "  batchsize = " + str(batch) +"   performance is " + str(np.mean(ann.eval_["valid_preform"])))
        print("---------------------------")

Sigmoid for nr of epochs 1 and eta: 0.0001  batchsize = 1   performance is 0.730092307692
Sigmoid for nr of epochs 1 and eta: 0.0001  batchsize = 10   performance is 0.729461538462
Sigmoid for nr of epochs 1 and eta: 0.0001  batchsize = 50   performance is 0.763861538462
---------------------------
Sigmoid for nr of epochs 1 and eta: 0.001  batchsize = 1   performance is 0.883276923077
Sigmoid for nr of epochs 1 and eta: 0.001  batchsize = 10   performance is 0.862507692308
Sigmoid for nr of epochs 1 and eta: 0.001  batchsize = 50   performance is 0.876076923077
---------------------------
Sigmoid for nr of epochs 1 and eta: 0.01  batchsize = 1   performance is 0.994276923077
Sigmoid for nr of epochs 1 and eta: 0.01  batchsize = 10   performance is 0.994907692308
Sigmoid for nr of epochs 1 and eta: 0.01  batchsize = 50   performance is 0.992476923077
---------------------------
Sigmoid for nr of epochs 1 and eta: 0.1  batchsize = 1   performance is 0.998892307692
Sigmoid for nr of epoc